In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import glob
import os
from joblib import Parallel, delayed 
import concurrent.futures
pd.options.mode.chained_assignment = None

In [33]:
def load_all_paths_parquet_with_text(country, year):
    path = '../data_with_text/' + country + '/' + year + '/*.parquet'
    return sorted(glob.glob(path))

def load_all_paths_parquet_no_text(country, year):
    path = '../data_no_text/' + country + '/' + year + '/*.parquet'
    return sorted(glob.glob(path))

def load_file(path):
    table = pq.read_table(path)
    return table.to_pandas()

def load_files(country, year, number = False):
    if number != False:
        paths = load_all_paths_parquet_no_text(country, year)[:number]
    else:
        paths = load_all_paths_parquet_no_text(country, year)
    myDataFrame = pd.DataFrame()
    for path in paths:
        table = pq.read_table(path).to_pandas()
        myDataFrame = myDataFrame.append(table)
    return myDataFrame

In [34]:
country = 'UK'

In [35]:
def saveDataByYear(year):
    country = 'UK'
    df = load_files(country, str(year))
    df = df[['JobID', 'SICCode','JobDate','CanonCity','CanonCounty',
         'LocalAuthorityDistrict',
         'CanonSkillClusters', 'SOCCode', 'CanonSkills']]
    df['date'] = pd.to_datetime(df['JobDate'])
    df.to_pickle(country + '_' + str(year) + '.pkl')
    
    

In [36]:
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     [executor.submit(saveDataByYear,year) for year in range(2014,2020)]

# The following command is running out of memory
# Parallel(n_jobs=-2, verbose=100)(delayed(saveDataByYear)(year,) for year in range(2012,2014))


for year in range(2012,2020):
    df = load_files(country, str(year))
    df = df[['JobID', 'SICCode','JobDate','CanonCity','CanonCounty',
         'LocalAuthorityDistrict',
         'CanonSkillClusters', 'SOCCode', 'CanonSkills']]
    df['date'] = pd.to_datetime(df['JobDate'])
    df.to_pickle(country + '_' + str(year) + '.pkl')

In [37]:
df = pd.read_pickle(country + '_' + str(2012) + '.pkl')
df

,JobID,SICCode,JobDate,CanonCity,CanonCounty,LocalAuthorityDistrict,CanonSkillClusters,SOCCode,CanonSkills,date
0,25380105,None,2012-01-01,MANCHESTER,Manchester,Manchester,Specialised Skills|Specialised Skills|Health C...,4138,"[Basic Internet Skills, Detail-Orientated, Wor...",2012-01-01
1,25380106,None,2012-01-01,LONDON,Greater London,None,Specialised Skills|Personal Care and Services:...,3562,"[Basic Internet Skills, Child Care, Communicat...",2012-01-01
2,25380107,78.10/9,2012-01-01,BRISTOL,City of Bristol,City of Bristol,Specialised Skills|Finance: Budget Management;...,3562,"[Basic Internet Skills, Budgeting, Business De...",2012-01-01
3,25380108,78.10/9,2012-01-01,HEMEL HEMPSTEAD,Hertfordshire,Dacorum,Specialised Skills,3562,[Basic Internet Skills],2012-01-01
4,25380109,78.10/9,2012-01-01,BIRMINGHAM,Birmingham,Birmingham,Specialised Skills|Specialised Skills,3562,"[Basic Internet Skills, Communication Skills]",2012-01-01
...,...,...,...,...,...,...,...,...,...,...
13949,19866318,69.10,2012-12-31,LONDON,Greater London,None,Specialised Skills|Finance: Billing and Invoic...,3520,"[Communication Skills, Customer Billing, Detai...",2012-12-31
13950,19866425,None,2012-12-31,LEICESTER,Leicester,Leicester,Finance: Accounts Payable and Receivable;Speci...,4122,"[Accounts Payable / Accounts Receivable, Micro...",2012-12-31
13951,663375863,None,2012-12-30,PETERBOROUGH,Peterborough,Peterborough,Information Technology: Technical Support;Spec...,3131,"[Application Support, Asset Management Industr...",2012-12-30
13952,663378519,None,2012-12-30,PETERBOROUGH,Peterborough,Peterborough,Finance: General Accounting;Specialised Skills...,1259,"[Accounting, Budgeting, Business Process, Cont...",2012-12-30
